# Otimizando Geometrias usando VASP e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

Importando variáveis do VASP

In [ ]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/elvis/Programs/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu' # usando VASP com GPU
# os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 4 vasp_std_cpu' # usando VASP com CPU
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

# Importando o VASP calculator do ASE
from ase.calculators.vasp import Vasp

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ase import Atoms, Atom
from ase.build import molecule
from ase.io import write, read
from ase.visualize.plot import plot_atoms
from ase.visualize import view

# Geometria de Moléculas

## Molécula de H2O


Uma opção para gerar geometria da molécula

In [ ]:
h2omol = Atoms([Atom('O', [0, 0, 0]),
                Atom('H', [0.0, -0.760265, 0.588373]),
                Atom('H', [0.0, 0.760265, 0.588373])])
h2omol.center(vacuum=4.0) # coloca dentro de uma caixa com 4 Angstroms de vácuo em todas as direções
h2omol.pbc = True # periodic boundary conditions

In [ ]:
fig, ax = plt.subplots()
plot_atoms(h2omol, ax, radii=0.3, rotation=('0x,90y,0z'))

In [ ]:
view(h2omol, viewer='x3d')

Calculando comprimendo da ligação

In [ ]:
l = h2omol.get_distance(0, 2)

print(f"O comprimento da ligação O-H é {l:.3f} Å")

Calculando o ângulo da ligação

In [ ]:
theta = h2omol.get_angle(2, 0, 1)

print(f"O ângulo da ligação H-O-H é {theta:.3f} graus")

ASE possui algumas moléculas já pré-definidas

In [ ]:
h2omol = molecule('H2O')        # molécula
h2omol.center(vacuum=4.0)       # ~8 Å box; 4 Å para cada lado, bom para isolar molécula
h2omol.pbc = True               # aplica condição de contorno periódica

In [ ]:
view(h2omol, viewer='x3d')

Calculadora do VASP

In [ ]:
calc = Vasp(xc='PBE',                                   # funcional
            encut=350,                                  # safe default for PAW-PBE sets
            kpts=[1, 1, 1],gamma=True,                  # Somente pontos Gamma em Fourier
            ibrion=-1,                                  # Calcula sem atualizar posições atômicas
            directory='moleculas/H2O'                   # pasta em que os cálculos serão armazenados
        )

In [ ]:
h2omol.calc = calc

In [ ]:
E0 = h2omol.get_potential_energy()       

print('Energia: {:.3f} eV'.format(E0))

In [ ]:
fig, ax = plt.subplots()
plot_atoms(h2omol, ax, radii=0.3, rotation=('0x,90y,0z'))

In [ ]:
print("Posições (Å):\n", h2omol.get_positions())

In [ ]:
l = h2omol.get_distance(0, 1)

print(f"O comprimento da ligação O-H é {l:.3f} Å")

In [ ]:
theta = h2omol.get_angle(2, 0, 1)

print(f"O ângulo da ligação H-O-H é {theta:.3f} graus")

### Variando parâmetros do VASP

Checando a convergência com o tamanho da caixa

In [ ]:
Larray = np.array([6.0, 8.0, 10.0, 12.0, 14.0]) # tamanhos da caixa
Earray = np.zeros(len(Larray))

for i, L in enumerate(Larray):
    h2omol.center(vacuum=L/2.0)    # box de tamanho L

    calc = Vasp(xc='PBE',                                   # funcional
                encut=350,                                  # safe default for PAW-PBE sets
                kpts=[1, 1, 1],gamma=True,                  # Somente pontos Gamma em Fourier
                ibrion=-1,                                  # Sem atualizar posições atômicas
                directory=f'moleculas/H2O/L-{int(L)}',       # pasta em que os cálculos serão armazenados
                atoms = h2omol                           
            )

    Earray[i] = h2omol.get_potential_energy()

    print(f"L = {L:.1f} Å, E = {Earray[i]:.3f} eV")

In [ ]:
plt.plot(Larray, Earray, 'o-')
plt.xlabel('Tamanho da caixa L (Å)')
plt.ylabel('Energia total (eV)')
plt.title('Convergência da energia da molécula de H2O com L')

Checando convergência com `ENCUT`

`ENCUT` especifica o cutoff de energia para o conjunto de base de ondas planas em eV. Todas as ondas planas com energia cinética menor que $E_\text{cut}$ serão incluídas no conjunto de base, i.e., 

$$|\boldsymbol{G} + \boldsymbol{k}| < G_\text{cut} \quad \text{com} \quad E_\text{cut} = \frac{\hbar^2}{2m} G_\text{cut}^2$$

Ref: https://www.vasp.at/wiki/ENCUT

In [ ]:
ENCUTarray = np.array([250,300,350,400,450,500]) # valores de ENCUT
Earray = np.zeros(len(ENCUTarray))

h2omol.center(vacuum=4.0)    # box de tamanho L

for i, encut in enumerate(ENCUTarray):
    
    calc = Vasp(xc='PBE',                                   # funcional
                encut=encut,                                # safe default for PAW-PBE sets
                kpts=[1, 1, 1],gamma=True,                  # Somente pontos Gamma em Fourier
                ibrion=-1,                                  # Sem atualizar posições atômicas
                directory=f'moleculas/H2O/encut-{encut}',   # pasta em que os cálculos serão armazenados
                atoms = h2omol                           
            )

    Earray[i] = h2omol.get_potential_energy()

    print(f"ENCUT = {encut:.1f} Å, E = {Earray[i]:.3f} eV")

In [ ]:
plt.plot(ENCUTarray, Earray, 'o-')
plt.xlabel('ENCUT (eV)')
plt.ylabel('Energia total (eV)')
plt.title('Convergência da energia da molécula de H2O com ENCUT')

### Otimizando a geometria da molécula de H2O

In [ ]:
THETAarray = np.array([90, 95, 100, 105, 110, 115, 120]) # valores de ângulos de ligação
Earray = np.zeros(len(THETAarray))

for i, theta in enumerate(THETAarray):
    h2omol.set_angle(2, 0, 1, theta)  # fixa o oxigênio e altera o ângulo H-O-H

    calc = Vasp(xc='PBE',                                   # funcional
                encut=350,                                  # safe default for PAW-PBE sets
                kpts=[1, 1, 1],gamma=True,                  # Somente pontos Gamma em Fourier
                ibrion=-1,                                  # Sem atualizar posições atômicas
                directory=f'moleculas/H2O/theta-{int(theta)}', # pasta em que os cálculos serão armazenados
                atoms = h2omol                           
            )

    Earray[i] = h2omol.get_potential_energy()

    print(f"theta = {theta:.1f}°, E = {Earray[i]:.3f} eV")

In [ ]:
plt.plot(THETAarray, Earray, 'or')
plt.xlabel('Ângulo de ligação H-O-H (°)')
plt.ylabel('Energia (eV)')
plt.title('Energia da molécula de H2O vs. ângulo de ligação')

Fitting parabólico

In [ ]:
z = np.polyfit(THETAarray, Earray, 2)

print(z)

p = np.poly1d(z)

In [ ]:
thetarray = np.arange(THETAarray[0], THETAarray[-1]+0., 0.1)

plt.plot(thetarray, p(thetarray), '-', label='Fitting parabólico')


plt.plot(THETAarray, Earray, 'or')
plt.xlabel('Ângulo de ligação H-O-H (°)')
plt.ylabel('Energia (eV)')
plt.title('Energia da molécula de H2O vs. ângulo de ligação')

### Forças nos núcleos atômicos

Sendo a energia total de uma molécula dada por $E[\rho(\boldsymbol{r})]$ calculada a partir do DFT por 
$$E[\rho(\boldsymbol{r})]= E_{HK}[\rho(\boldsymbol{r})] - \sum_{A=1}^M \int \frac{Z_A}{|\boldsymbol{r}-\boldsymbol{R_A}|} \rho(\boldsymbol{r})\ \text{d} \boldsymbol{r} + \frac{1}{2}\sum_{A=1}^M \sum_{B\neq A}^M \frac{Z_A Z_B}{|\boldsymbol{R}_B-\boldsymbol{R_A}|} $$
de tal forma que a força experimentada por um núcleo $C$ seja dada por 
$$\boldsymbol{F}_C = -\frac{\partial E}{\partial \boldsymbol{R}_C}$$
que é uma consequência do **teorema de Hellmann-Feynman**. Usando o DFT podemos calcular essa força a partir de 
$$ \frac{\partial E}{\partial \boldsymbol{R}_C}=\frac{\partial }{\partial \boldsymbol{R}_C}\left[- \sum_{A=1}^M \int \frac{Z_A}{|\boldsymbol{r}-\boldsymbol{R_A}|} \rho(\boldsymbol{r})\ \text{d} \boldsymbol{r} + \frac{1}{2}\sum_{A=1}^M \sum_{B\neq A}^M \frac{Z_A Z_B}{|\boldsymbol{R}_B-\boldsymbol{R_A}|} \right]$$
de modo que a força experimentada pelos núcleos atômicos é proporcional ao campo elétrico gerado pelos elétrons e pelos demais núcleos. Com isso, chegamos a 
$$ \frac{\partial E}{\partial \boldsymbol{R}_C}=- \int \frac{Z_C (\boldsymbol{r}-\boldsymbol{R_C})}{|\boldsymbol{r}-\boldsymbol{R_C}|^3} \rho(\boldsymbol{r})\ \text{d} \boldsymbol{r} + \sum_{A\neq C}^M \frac{Z_A Z_C (\boldsymbol{R}_A-\boldsymbol{R_C})}{|\boldsymbol{R}_A-\boldsymbol{R_C}|^3} $$


No VASP, os métodos de otimização de estrutura são definidos por 
- `ibrion=-1` Não atualiza a posição dos núcleos
- `ibrion=1` RMM-DIIS (combinação linear das posições anteriores, mais rapidos)
- `ibrion=2` conjugate gradient (mais robusto)
- `ibrion=3` damped molecular dynamics (MD com velocidade diminuindo, para sistemas grandes e longe do equilíbrio)

REF: https://www.vasp.at/wiki/Structure_optimization

In [ ]:
h2omol = molecule('H2O')        # molécula
h2omol.center(vacuum=4.0)      # ~8 Å box; 4 Å para cada lado, bom para isolar molécula
h2omol.pbc = True               # aplica condição de contorno periódica

In [ ]:
calc = Vasp(directory='moleculas/H2O/relaxed',
            xc='PBE',                                   # funcional
            encut=350,                                  # safe default for PAW-PBE sets
            kpts=[1, 1, 1],gamma=True,                  # k-points
            ismear=0, sigma=0.05,                       # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02,                   # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,                  # CG ionic relax; fixed cell
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            atoms=h2omol)

In [ ]:
E_relax = h2omol.get_potential_energy()       

print('Energia: {:.3f} eV'.format(E_relax))

In [ ]:
view(h2omol, viewer='x3d')

Exportando geometria otimizada

In [ ]:
write('H2O_relaxed.pdb', h2omol)

In [ ]:
print("Número de passos executados:", calc.get_number_of_iterations())

In [ ]:
print("Forças em ev/A")
print("=======")
print(h2omol.get_forces())

In [ ]:
l = h2omol.get_distance(0, 1)

print(f"O comprimento da ligação O-H é {l:.3f} Å")

In [ ]:
theta = h2omol.get_angle(2, 0, 1)

print(f"O ângulo da ligação H-O-H é {theta:.3f} graus")

### Analisando Orbitais Moleculares

Determinando as energias dos orbitais moleculares

In [ ]:
calc.get_eigenvalues()

Determinando as ocupações

In [ ]:
calc.get_occupation_numbers()

Determinando as energias dos orbitais HOMO e LUMO

In [ ]:
homo, lumo = calc.get_homo_lumo()

print(f"Lumo: {lumo:.3f} eV")
print(f"Homo: {homo:.3f} eV")
print('----------')
print(f"Diferença: {lumo-homo:.3f} eV")

In [ ]:
gap = (lumo - homo)

# Eigenvalues and occupations (Kohn–Sham levels)
eigs = np.array(calc.get_eigenvalues())
occ  = np.array(calc.get_occupation_numbers())

# Sort for a clean plot
order = np.argsort(eigs)
eigs, occ = eigs[order], occ[order]

# Split occupied / unoccupied
occ_mask = occ > 0.5
E_occ = eigs[occ_mask]
E_unocc = eigs[~occ_mask]

# --- plotting (single chart, no explicit colors) ---
plt.figure(figsize=(3, 3.2), dpi=140)

# Sticks for occupied and unoccupied states
for E in E_occ:
    plt.plot([0, 0.6], [E, E],color='C0')
for E in E_unocc:
    plt.plot([0.7, 1.3], [E, E],color='grey')

# Highlight HOMO / LUMO
plt.plot([0, 1.3],[homo, homo],color='red', linewidth=2)
plt.plot([0, 1.3],[lumo, lumo],color='red', linewidth=2)

# Annotate
plt.text(1.6,homo-1, 'HOMO', ha='center', va='bottom', fontsize=9)
plt.text(1.6, lumo-1, 'LUMO', ha='center', va='bottom', fontsize=9)
plt.title(f'HOMO-LUMO gap = {gap:.3f} eV')

plt.ylabel('Energia (eV)')
plt.xticks([])          # hide y-axis (pure spectrum)
plt.ylim(min(eigs)-1, max(eigs)+1)
plt.grid(True, axis='y', linestyle=':', linewidth=0.7)
plt.tight_layout()

print(f"HOMO = {homo:.3f} eV, LUMO = {lumo:.3f} eV, GAP = {gap:.3f} eV")

In [ ]:
print(f'Comprimento de onda da radiação pro gap é de {1239.8/gap:.3f} nm')

### Adicionando interação de van der Waals

$$E_\text{DFT+vDW} = E_\text{KS} - \frac{1}{2}\sum_{A=1}^M \sum_{B \neq A}^M \frac{C_{AB}}{|\boldsymbol{R}_A-\boldsymbol{R}_B|^6}$$

com $C_{AB} = (C_A C_B)^{1/2}$ é o coeficiente de dispersão calculado pela fórmula de Casimir-Polder

$$C_{AB} = \frac{3}{\pi}\int_0^\infty \alpha_A(i \omega ) \alpha_B (i \omega)\ \text{d}\omega$$

com $\alpha_i$ sendo a polarizabilidade do átomo $i$. 

No VASP, basta fazer `ivdw=12` no `calc` afim de utilizar a correção de Becke-Johnson (BJ)

REF: https://www.vasp.at/wiki/index.php/DFT-D3

In [ ]:
calc_vdw = Vasp(directory="moleculas/H2O/vdw",
            xc="PBE",
            encut=350,
            kpts=[1, 1, 1],gamma=True,
            ismear=0, sigma=0.05,     # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02, # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=2, nsw=150,# CG ionic relax; fixed cell
            ivdw=12,                  # D3(BJ) dispersion (recommended for molecules)
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            atoms=h2omol)

In [ ]:
E_vdw = h2omol.get_potential_energy()   

print('Energia com correção de vdW: {:.3f} eV'.format(E_vdw))

In [ ]:
print(E_vdw-E_relax, ' eV')

In [ ]:
homo, lumo = calc_vdw.get_homo_lumo()

gap = lumo-homo

print(f"Lumo: {lumo:.3f} eV")
print(f"Homo: {homo:.3f} eV")
print('----------')
print(f"Diferença: {gap:.3f} eV")

In [ ]:
print(f'Comprimento de onda da radiação pro gap é de {1239.8/gap:.3f} nm')

## Molécula de Ciclopentano

In [ ]:
# Lendo arquivo XYZ
Ciclopent_molecule = read('structures/ciclopentane.xyz') 
Ciclopent_molecule.center(vacuum=4.0) 
Ciclopent_molecule.pbc = True 

# See basic info
print(Ciclopent_molecule)
print('Número de átomos:', len(Ciclopent_molecule))
print("Cell:", Ciclopent_molecule.get_cell())

In [ ]:
view(Ciclopent_molecule, viewer='x3d')

In [ ]:
calc = Vasp(
            directory='moleculas/ciclopentano/relaxed',
            xc='PBE',
            encut=520,                          # safe PAW-PBE default
            kpts=[1, 1, 1], gamma=True,
            ismear=0, sigma=0.05,               # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02,           # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,          # CG ionic relax; fixed cell
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            # Optional dipole correction (often helpful for isolated polar molecules):
            # ldipol=True, idipol=3
            atoms=Ciclopent_molecule
        )

In [ ]:
E_relax = Ciclopent_molecule.get_potential_energy()       

print('Energia: {:.3f} eV'.format(E_relax))

Exportando geometria otimizada

In [ ]:
write('ciclopentano_relaxed.pdb', Ciclopent_molecule)

In [ ]:
calc_vdw = Vasp(
            directory='moleculas/ciclopentano/vdw',
            xc='PBE',
            encut=520,                # safe PAW-PBE default
            kpts=[1, 1, 1], gamma=True,
            ismear=0, sigma=0.05,     # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02, # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,# CG ionic relax; fixed cell
            ivdw=12,                  # D3(BJ) dispersion (recommended for molecules)
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            # Optional dipole correction (often helpful for isolated polar molecules):
            # ldipol=True, idipol=3
            atoms=Ciclopent_molecule
        )

In [ ]:
E_vdw = Ciclopent_molecule.get_potential_energy()   

print('Energia com correção de vdW: {:.3f} eV'.format(E_vdw))

In [ ]:
print(E_vdw-E_relax, ' eV')

---

# Geometria de Sólidos

## Cristal de Cu

Criando a partir dos vetores da base do FCC

In [ ]:
a = 2.53
Cu_crystal = Atoms([Atom("Cu", (0, 0, 0))],
                cell=0.5 * a * np.array([[1.0, 1.0, 0.0],
                [0.0, 1.0, 1.0],
                [1.0, 0.0, 1.0]]), pbc=True)

print(Cu_crystal)
print("Cell:", Cu_crystal.get_cell())
print("Positions:\n", Cu_crystal.get_positions())

In [ ]:
view(Cu_crystal, viewer='x3d')

Criando a partir de arquivo CIF

Ref: https://legacy.materialsproject.org/materials/mp-30/

In [ ]:
# Lendo arquivo CIF
Cu_crystal = read('structures/Cu.cif') # célula convencional

# See basic info
print(Cu_crystal)
print("Cell:", Cu_crystal.get_cell())
print("Positions:\n", Cu_crystal.get_positions())

Cu_crystal.get_number_of_atoms()

In [ ]:
view(Cu_crystal, viewer='x3d')

In [ ]:
calc = Vasp(directory='solidos/Cu',
                xc='PBE',
                kpts=[6, 6, 6],  # specifies k-points
                encut=350,
                atoms = Cu_crystal)

In [ ]:
E_Cu = Cu_crystal.get_potential_energy()

print(f'Energia total do cristal de Cu: {E_Cu:.3f} eV')

In [ ]:
E_per_Cu = E_Cu / len(Cu_crystal)

print(f'Energia por átomo de Cu {E_per_Cu:.3f} eV')

### Otimizando parâmetros do VASP

Variando número de `kpoints`

In [ ]:
KPTSarray = [2, 3, 4, 5, 6, 8, 10] # valores de k-points
Earray = np.zeros(len(KPTSarray))

for i, k in enumerate(KPTSarray):
    
    calc = Vasp(directory='solidos/Cu/kpoints-{0}'.format(k),
                xc='PBE',
                kpts=[k, k, k],  # specifies k-points
                encut=350,
                ibrion=-1,                                  # Sem atualizar posições atômicas
                atoms=Cu_crystal)

    Earray[i] = Cu_crystal.get_potential_energy()

    print(f"KPOINTS = {k:.1f}, E = {Earray[i]:.3f} eV")

In [ ]:
plt.plot(KPTSarray, Earray, 'o-')
plt.xlabel('Número de k-points em cada direção')
plt.ylabel('Energia (eV)')
# plt.ylim(-15,-14.8)

Variando o parâmetro `encut`

In [ ]:
ENCUTarray = np.array([250,300,350,400,450,500]) # valores de ENCUT
Earray = np.zeros(len(ENCUTarray))

for i, encut in enumerate(ENCUTarray):
    
    calc = Vasp(directory='solidos/Cu/encut-{0}'.format(encut),
                xc='PBE',
                kpts=[6, 6, 6],                        
                encut=encut,
                ibrion=-1,                                  # Sem atualizar posições atômicas
                atoms=Cu_crystal)

    Earray[i] = Cu_crystal.get_potential_energy()

    print(f"ENCUT = {encut:.1f} Å, E = {Earray[i]:.3f} eV")

In [ ]:
plt.plot(ENCUTarray, Earray,'o-')
plt.xlabel('ENCUT (eV)')
plt.ylabel('Energia (eV)')

### Otimizando o tamanho da célula cristalina

In [ ]:
Cu_crystal.get_cell()

In [ ]:
a_cell = np.array([ 3.2, 3.4, 3.6, 3.8, 4.0,4.2]) # valores em Angstrons
E_total = np.zeros_like(a_cell)

for i, a in enumerate(a_cell):
    Cu_crystal.set_cell([a, a, a], scale_atoms=True)

    calc = Vasp(directory='solidos/Cu/a-{:.1f}'.format(a),
                xc='PBE',
                kpts=[6, 6, 6],  # specifies k-points
                encut=350,
                atoms = Cu_crystal)
    
    E = Cu_crystal.get_potential_energy()
    E_total[i] = E
    print(f'a = {a: .2f} Å  -->  E = {E: .6f} eV')

In [ ]:
plt.plot(a_cell, E_total, 'or')
plt.xlabel('a (Å)')
plt.ylabel('Energia (eV)')
plt.title('Energia do Cristal de Cu vs. Parâmetro de Rede a')

Fitting de polinômio quadrático

In [ ]:
z = np.polyfit(a_cell, E_total, 2)
p = np.poly1d(z)

#### Equação de Estado de Murnaghan

$$E(V) = E_0 + \frac{B_0 V}{B_0'}\left[\frac{(V_0/V)^{B_0'}}{B_0'-1}+1 \right] - \frac{B_0 V}{B_0'-1}$$

com 
- $V$: volume da célula unitária
- $V_0$: volume da célula unitária com energia mínima
- $B_0$: módulo de compressibilidade $B_0 = - V (\partial P /\partial V)_T$.
- $B_0'$: derivada do módulo de compressibilidade com a pressão

*Ref:* C. L. Fu and K. M. Ho. First-principles calculation of the equilibrium ground-state properties of transition metals: Applications to Nb and Mo. Phys. Rev. B, 28:5480–5486, Nov 1983. doi: [10.1103/PhysRevB.28.5480](http://link.aps.org/doi/10.1103/PhysRevB.28.5480).

In [ ]:
def Murnaghan(a, a0, E0, B0, BP):
    vol = a**3
    V0 = a0**3
    return E0 + (B0*vol/BP)*(((V0/vol)**BP)/(BP-1)+1) - V0*B0/(BP-1)


from scipy.optimize import curve_fit

popt, pcov = curve_fit(Murnaghan, a_cell, E_total, p0=[a_cell[E_total.argmin()],E_total.min(),0.0,2.0])

a0 = popt[0]
E0 = popt[1]
B0 = popt[2]
BP = popt[3]

print(f'Fitted parameters: a0={a0:.4f} Å, E0={E0:.4f} eV, B0={B0:.4f} eV/Å³, BP={BP:.4f}')

In [ ]:
a_array = np.arange(a_cell[0], a_cell[-1]+0.01, 0.01)

plt.plot(a_cell, E_total, 'or')
plt.plot(a_array, p(a_array), '--',label='Quadrática')
plt.plot(a_array, Murnaghan(a_array,*popt), '-k',label='Murnaghan')
plt.xlabel('a (Å)')
plt.ylabel('Energia (eV)')
plt.title('Energia do Cristal de Cu vs. Parâmetro de Rede a')
plt.legend(loc='best')

Calculando energia para a geometria otimizada

In [ ]:
Cu_crystal.set_cell([a0, a0, a0], scale_atoms=True)

calc = Vasp(directory='solidos/Cu',
                xc='PBE',
                kpts=[6, 6, 6],  # specifies k-points
                encut=300,
                atoms = Cu_crystal)

In [ ]:
E_Cu = Cu_crystal.get_potential_energy()

print(f'Energia total do cristal de Cu: {E_Cu:.3f} eV')

E_per_Cu = E_Cu / len(Cu_crystal)

print(f'Energia por átomo de Cu {E_per_Cu:.3f} eV')

### Otimizando geometria direto no VASP

In [ ]:
# Lendo arquivo CIF
Cu_crystal = read('structures/Cu.cif') # célula convencional

Cu_crystal.get_cell()

In [ ]:
calc_eq = Vasp(directory='solidos/Cu/relaxed',
                xc='PBE',
                kpts=[6, 6, 6],  # specifies k-points
                encut=350,
                prec='High',
                isif=3,
                nsw=10,
                ibrion=2,
                atoms=Cu_crystal
                )

In [ ]:
E_Cu = Cu_crystal.get_potential_energy()

print(f'Energia total do cristal de Cu: {E_Cu:.3f} eV')

E_per_Cu = E_Cu / len(Cu_crystal)

print(f'Energia por átomo de Cu {E_per_Cu:.3f} eV')

In [ ]:
print("Número de passos executados:", calc_eq.get_number_of_iterations())

In [ ]:
print("Cell:", Cu_crystal.get_cell())

In [ ]:
a0

Exportando arquivo `xyz`

In [ ]:
write('Cu_relaxed.xyz', Cu_crystal)